In [1]:
import torch
from t3_karpathy.compressing_transformer import CompressingAccumulativeTrainer
from t3_karpathy.transformer_config import TransformerConfig
from t3_karpathy.gpt_nano_dataloader import GptNanoDataloader
from t3_karpathy.karpathy_transformer import KarpathyRunner



In [ ]:

config = TransformerConfig()
dataloader = GptNanoDataloader(config)
device = 'cuda'

trainer1=CompressingAccumulativeTrainer(config)
#rainer1=KarpathyRunner(config)
trainer1.train_iterate(5000, dataloader.get_train_batch, dataloader.get_val_batch)

print(dataloader.token_codec.decode(trainer1.generate(torch.zeros((1, 1), dtype=torch.long, device=device), 100)[0].tolist()))

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(dataloader.token_codec.decode(trainer1.generate(context, max_new_tokens=2000)[0].tolist()))

File already downloaded
in_size=32, in_embd=64, out_size=16, out_embd=64
in_size=16, in_embd=64, out_size=8, out_embd=64
in_size=8, in_embd=64, out_size=16, out_embd=64
in_size=16, in_embd=64, out_size=32, out_embd=64
0.414218 M parameters
step 0: train loss 15.9134, val loss 15.9326
step 100: train loss 3.5526, val loss 3.5994
step 200: train loss 3.4700, val loss 3.5172
